In [92]:
from bs4 import BeautifulSoup
from pyquery import PyQuery as pq
from collections import defaultdict 
import time
import requests

### Scrape the movie gross Information from boxofficemojo.com 

In [93]:
pages = xrange(1,9)
pagestext = {}
for page in pages: 
    r = requests.get("http://www.boxofficemojo.com/yearly/chart/?page=%s&view=releasedate&view2=domestic&yr=2014&p=.htm"%page)
    pagestext[page] = r.text
    time.sleep(1)

In [94]:
movie_budget = defaultdict(list) 
year = 2014
for page in pages: 
    soup = BeautifulSoup(pagestext[page], "html.parser")
    rows = soup.find_all("font", attrs={'size':'2'})
           
    start = 10 
    for i in range(start,len(rows)-2):
        t = rows[i].get_text()
        if unicode('Summary of') in t: 
            break
        elif (i-start) % 9 == 0: 
            movie_budget['rank'].append(t)
        elif (i-start) % 9 == 1: 
            r = '('+str(year)+')'
            if unicode(r) in t: 
                j = t.index(unicode(r))
                movie_budget['title'].append(t[:j])
            else: 
                movie_budget['title'].append(t)
        elif (i-start) % 9 == 2: 
            movie_budget['studio'].append(t)
        elif (i-start) % 9 == 3: 
            movie_budget['gross'].append(t)
        elif (i-start) % 9 == 4: 
            movie_budget['gross theaters'].append(t)
        elif (i-start) % 9 == 5: 
            movie_budget['opening'].append(t)
        elif (i-start) % 9 == 6: 
            movie_budget['opening theaters'].append(t)
        elif (i-start) % 9 == 7: 
            movie_budget['open'].append(t)
        elif (i-start) % 9 == 8: 
            movie_budget['close'].append(t)
            
movie_budget['year'] = [year for i in range(len(movie_budget['title']))]

###Scrape movie budget information from the-numbers.com 

In [59]:
r_numbers = requests.get("http://www.the-numbers.com/movie/budgets/all")
rn_text = r_numbers.text 

In [60]:
d_=pq(rn_text)
d_tables=pq(d_('table'))
rows = pq(d_tables[0])('tr')

In [89]:
budget = defaultdict(list)
for j in range(1,len(rows)):
    dat = pq(rows[j])('td')
    for i in range(len(dat)): 
        if i % 6 == 1:
            budget['year'].append(int(pq(dat[i])('a').text().split('/')[-1]))
        elif i % 6 == 2:
            budget['title'].append(pq(dat[i])('a').text())
        elif i % 6 == 3: 
            budget['budget'].append(pq(dat[i]).text())